In [1]:
import os 
import cv2
import numpy as np
import clip
import torch
from PIL import Image
from tqdm import tqdm
import pathlib
from pathlib import Path
import glob
import argparse
import json
import jsonlines
import json_numpy
import pandas as pd
import pyarrow as pa
import time

/scratch/bbki/kastanday/conda_envs/envs/v3_clip_preprocessing_yt1b/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
table = pa.Table().from_pydict({'a': [1, 2, 3], 'b': [4, 5, 6]})


print(table)

TypeError: Do not call Table's constructor directly, use one of the `Table.from_*` functions instead.

In [79]:
video_filepath = '/tmp/hello.py'
i = 9
# one_segment_output = pa.Table.from_pandas(pd.DataFrame({
#                 "video_stem": [str(Path(video_filepath).stem), 'hi'],
#                 "segment_id": [str(Path(video_filepath).stem) + f"_{i}", 'hi'],
#                 "segment_index": [i, i],
# }))

# df = pd.DataFrame()
list_ofTihgsn = []
for i in range(5):
  one_segment_output = pd.DataFrame({
                  "video_stem": "VERSION 2",
                  "segment_id": "VERSION_2" + f"_{i}",
                  "segment_index": 69,
  }, index=[0])
  list_ofTihgsn.append(one_segment_output)

segs = pd.concat(list_ofTihgsn, ignore_index=True)
fastparquet.write('outfile.parq',segs, append=True)

In [54]:
two_segment_output = {
                "video_stem": str(Path(video_filepath).stem),
                "segment_id": str(Path(video_filepath).stem) + f"_{i}",
                "segment_index": i,
}
one_segment_output = {
                "video_stem": "str(Path(video_filepath).stem)",
                "segment_id": "str(Path(video_filepath).stem)" + f"_{i}",
                "segment_index": 10,
}

In [55]:
list_of_segment_outputs = [one_segment_output, two_segment_output]

In [67]:
# one_video_of_segments = pa.concat_tables(*list_of_segment_outputs)
one_video_of_segments = pa.Table.from_pylist(list_of_segment_outputs)
one_video_of_segments

pyarrow.Table
video_stem: string
segment_id: string
segment_index: int64
----
video_stem: [["str(Path(video_filepath).stem)","hello"]]
segment_id: [["str(Path(video_filepath).stem)_9","hello_9"]]
segment_index: [[10,9]]

In [63]:
import pyarrow.parquet as pq

In [80]:
from fastparquet import ParquetFile
pf = ParquetFile('outfile.parq')
df = pf.to_pandas()
df

,video_stem,segment_id,segment_index
index,,,
0,str(Path(video_filepath).stem),str(Path(video_filepath).stem)_9,9
0,VERSION 2,VERSION_2_9,69
0,VERSION 2,VERSION_2_0,69
1,VERSION 2,VERSION_2_1,69
2,VERSION 2,VERSION_2_2,69
3,VERSION 2,VERSION_2_3,69
4,VERSION 2,VERSION_2_4,69


In [81]:
list_of_filepaths = list(df['segment_id'])
list_of_filepaths

['str(Path(video_filepath).stem)_9',
 'VERSION_2_9',
 'VERSION_2_0',
 'VERSION_2_1',
 'VERSION_2_2',
 'VERSION_2_3',
 'VERSION_2_4']

In [75]:
import fastparquet

fastparquet.write('outfile.parq',one_segment_output, append=True)

# pyarrow.parquet.write_table('test_parquet_append.parquet', one_video_of_segments, append=False, schema=None, index=None)

In [2]:
video_data_path = '/scratch/bbki/kastanday/whisper/parallel_1'
video_input_dir = Path(video_data_path)
output_path = str(os.path.join(video_input_dir.parent, video_input_dir.stem + '_clip_output.jsonl'))
audio_jsonl = str(os.path.join(video_input_dir.parent, video_input_dir.stem + '_whisper_output.jsonl'))
audio_jsonl

'/scratch/bbki/kastanday/whisper/parallel_14_whisper_output.jsonl'

In [5]:
debug = True
video_dir_files = set()
stem_to_filename = {}
start_time = time.monotonic()

# GLOB VIDEOS
# for video_dir_file in glob.glob(os.path.join(video_data_path, '*'), recursive = True):
#     # print(Path(video_dir_file).stem)
#     video_dir_files.add(str(Path(video_dir_file).stem))
#     stem_to_filename[str(Path(video_dir_file).stem)] = video_dir_file
# print(f"Globbing from scratch 25k video files: {time.monotonic() - start_time}:2f seconds")

# FIND WHISPER JSON OBJS
start_time = time.monotonic()
audio_file_stems = []
stem_to_whisper = {}  # filename.stem --> whisper_json_object
one_json = None

df = pd.DataFrame()
with jsonlines.open(audio_jsonl) as reader:
    try:
        for idx, obj in enumerate(reader):
            json_objs = json.loads(obj)
            for json_obj in json_objs:
                audio_file_stems.append(json_obj['video_filename_stem'])

                if json_obj['video_filename_stem'] not in stem_to_whisper:
                    stem_to_whisper[json_obj['video_filename_stem']] = []
                
                stem_to_whisper[json_obj['video_filename_stem']].append(json_obj)
                
                one_json = json_obj
                
                df['video_filename_stem'] = json_obj
                if idx > 0:
                    break
    except Exception as e:
        print(f"Error: couldn't read {audio_jsonl}. Got error: {e}")
if debug:
    print(f"Done collecting {audio_jsonl}")

print(f"JsonL parsing time: {time.monotonic() - start_time:2f} seconds")
# stem_to_whisper
df

Done collecting /scratch/bbki/kastanday/whisper/parallel_14_whisper_output.jsonl
JsonL parsing time: 34.829023 seconds


,video_filename_stem
caption,I'M GOING ON EVERYONE HOW ARE YOU GUYS DOING O...
start,4.008
end,13.631562
segment_word_list,"[{'word': 'I'M', 'start': 4.008, 'end': 4.3286..."
video_filename_stem,u5DEHtDBqeI_The Grooming Guy_1076_got2b Shampoo
video_filepath,/tmp/parallel_14/u5DEHtDBqeI_The Grooming Guy_...


In [17]:
one_json['np_test'] = [1] * 512
one_json['segment_word_list'] = 'penis'

In [18]:
# thing  =np.zeros((512))

df2 = pd.DataFrame(one_json)
df2

,caption,start,end,segment_word_list,video_filename_stem,video_filepath,np_test
0,I'M GOING ON EVERYONE HOW ARE YOU GUYS DOING O...,4.008,13.631562,penis,u5DEHtDBqeI_The Grooming Guy_1076_got2b Shampoo,/tmp/parallel_14/u5DEHtDBqeI_The Grooming Guy_...,1
1,I'M GOING ON EVERYONE HOW ARE YOU GUYS DOING O...,4.008,13.631562,penis,u5DEHtDBqeI_The Grooming Guy_1076_got2b Shampoo,/tmp/parallel_14/u5DEHtDBqeI_The Grooming Guy_...,1
2,I'M GOING ON EVERYONE HOW ARE YOU GUYS DOING O...,4.008,13.631562,penis,u5DEHtDBqeI_The Grooming Guy_1076_got2b Shampoo,/tmp/parallel_14/u5DEHtDBqeI_The Grooming Guy_...,1
3,I'M GOING ON EVERYONE HOW ARE YOU GUYS DOING O...,4.008,13.631562,penis,u5DEHtDBqeI_The Grooming Guy_1076_got2b Shampoo,/tmp/parallel_14/u5DEHtDBqeI_The Grooming Guy_...,1
4,I'M GOING ON EVERYONE HOW ARE YOU GUYS DOING O...,4.008,13.631562,penis,u5DEHtDBqeI_The Grooming Guy_1076_got2b Shampoo,/tmp/parallel_14/u5DEHtDBqeI_The Grooming Guy_...,1
...,...,...,...,...,...,...,...
507,I'M GOING ON EVERYONE HOW ARE YOU GUYS DOING O...,4.008,13.631562,penis,u5DEHtDBqeI_The Grooming Guy_1076_got2b Shampoo,/tmp/parallel_14/u5DEHtDBqeI_The Grooming Guy_...,1
508,I'M GOING ON EVERYONE HOW ARE YOU GUYS DOING O...,4.008,13.631562,penis,u5DEHtDBqeI_The Grooming Guy_1076_got2b Shampoo,/tmp/parallel_14/u5DEHtDBqeI_The Grooming Guy_...,1
509,I'M GOING ON EVERYONE HOW ARE YOU GUYS DOING O...,4.008,13.631562,penis,u5DEHtDBqeI_The Grooming Guy_1076_got2b Shampoo,/tmp/parallel_14/u5DEHtDBqeI_The Grooming Guy_...,1
510,I'M GOING ON EVERYONE HOW ARE YOU GUYS DOING O...,4.008,13.631562,penis,u5DEHtDBqeI_The Grooming Guy_1076_got2b Shampoo,/tmp/parallel_14/u5DEHtDBqeI_The Grooming Guy_...,1


In [ ]:
all_whisper = []
with jsonlines.open(audio_jsonl) as reader:
    try:
        for _, obj in enumerate(reader):
            json_objs = json.loads(obj)
    except Exception as e:
        print(f"Error: couldn't read {audio_jsonl}. Got error: {e}")

In [30]:
from pyarrow import json
import pandas as pd
start_time = time.monotonic()
# table = json.read_json(str(audio_jsonl))
# table = jsonlines.read_json(str('small.jsonl'))
df2 = pd.read_json(str(audio_jsonl), lines=True, encoding='utf-8')
print(f"JsonL --> pyarrow parsing time: {time.monotonic() - start_time:2f} seconds")
df2

JsonL --> pyarrow parsing time: 15.658412 seconds


,0
0,"[{""caption"": ""THE NEW AT FIVE A STUDENT GOT SE..."
1,"[{""caption"": ""I DIDN'T KNOW WHAT TO GET ANYONE..."
2,"[{""caption"": ""IT'S TIME FOR A JOKE A JOKE FROM..."
3,"[{""caption"": ""HEY Y'ALL MY NAME IS RAVANNA AND..."
4,"[{""caption"": ""A BRAND NEW SEASON OF JAYLON'S B..."
...,...
19558,"[{""caption"": ""WE'RE HERE TODAY BECAUSE WE'VE B..."
19559,"[{""caption"": ""I'M GOING TO GO BACK TO THERE WI..."
19560,"[{""caption"": ""I'M SORRY MAJOR DO YOU BRING ANY..."
19561,"[{""caption"": ""SO WELCOME BACK TO THE WIFE MOTH..."


In [28]:
df2[0]


'[{"caption": "THE NEW AT FIVE A STUDENT GOT SENT HOME FROM ROSDALE MIDDLE SCHOOL AFTER BRINGING", "start": 0.0601875, "end": 4.2144375, "segment_word_list": [{"word": "THE", "start": 0.0601875, "end": 0.1404375}, {"word": "NEW", "start": 0.1805625, "end": 0.32099999999999995}, {"word": "AT", "start": 0.3611875, "end": 0.4414375}, {"word": "FIVE", "start": 0.481625, "end": 0.8026875}, {"word": "A", "start": 0.842875, "end": 0.9030625}, {"word": "STUDENT", "start": 0.94325, "end": 1.284375}, {"word": "GOT", "start": 1.3044375, "end": 1.4649375}, {"word": "SENT", "start": 1.4850625, "end": 1.6857499999999999}, {"word": "HOME", "start": 1.7259375, "end": 1.9266249999999998}, {"word": "FROM", "start": 1.96675, "end": 2.12725}, {"word": "ROSDALE", "start": 2.1674375, "end": 2.5888750000000003}, {"word": "MIDDLE", "start": 2.6290625, "end": 2.8498124999999996}, {"word": "SCHOOL", "start": 2.8899375, "end": 3.1708749999999997}, {"word": "AFTER", "start": 3.6124375, "end": 3.813125}, {"word": 

In [9]:
!head -n1 '/scratch/bbki/kastanday/whisper/parallel_13_whisper_output.jsonl'

"[{\"caption\": \"HOMEWARD BOUND IS A PROGRAM THAT WAS STARTED OUT OF AUSTRALIA AND ITS INTERNATIONAL LEADERSHIP\", \"start\": 2.247625, \"end\": 7.2059999999999995, \"segment_word_list\": [{\"word\": \"HOMEWARD\", \"start\": 2.247625, \"end\": 2.5285625}, {\"word\": \"BOUND\", \"start\": 2.548625, \"end\": 3.251}, {\"word\": \"IS\", \"start\": 3.351375, \"end\": 3.4516875}, {\"word\": \"A\", \"start\": 3.491875, \"end\": 3.5721249999999998}, {\"word\": \"PROGRAM\", \"start\": 3.873125, \"end\": 4.1540625}, {\"word\": \"THAT\", \"start\": 4.1741875, \"end\": 4.254437500000001}, {\"word\": \"WAS\", \"start\": 4.2745, \"end\": 4.35475}, {\"word\": \"STARTED\", \"start\": 4.3949375, \"end\": 4.655812500000001}, {\"word\": \"OUT\", \"start\": 4.6959375, \"end\": 4.796250000000001}, {\"word\": \"OF\", \"start\": 4.8364375, \"end\": 4.896624999999999}, {\"word\": \"AUSTRALIA\", \"start\": 5.1374375, \"end\": 5.6993125}, {\"word\": \"AND\", \"start\": 5.7194375, \"end\": 5.7996875}, {\"word\"